# **Proyecto MOS - Etapa 2**

In [ ]:
import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"
    import helper
    helper.install_idaes()
    helper.install_glpk()
    helper.install_ipopt()

!pip install networkx matplotlib
!pip install haversine folium
!pip install amplpy pyomo -q
!python -m amplpy.modules install coin highs scip gcg -q

from pyomo.environ import *
from haversine import haversine
import pandas as pd
import requests

import folium
import sympy as sp
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
from amplpy import modules
import matplotlib.pyplot as plt

--2025-05-04 05:34:18--  https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7171 (7.0K) [text/plain]
Saving to: ‘helper.py.75’

helper.py.75        100%[===================>]   7.00K  --.-KB/s    in 0s      

2025-05-04 05:34:18 (53.6 MB/s) - ‘helper.py.75’ saved [7171/7171]

IDAES found! No need to install.
Installing glpk via apt-get...
Ipopt found! No need to install.
ipopt was successfully installed
k_aug was successfuly installed
cbc was successfuly installed
clp was successfuly installed
bonmin was successfuly installed
couenne was successfuly installed
ipopt_l1 was successfuly installed
 


## **Modelo Matemático**

### **Suposiciones**
* La demanda de los clientes y la capacidad de los centros de distribución estan en las mismas unidades.
* Para los casos a resolver, los nodos no están concentrados en un único sector.
* Los vehículos deben cargar inventario desde algún centro de distribución al comenzar la jornada y deben regresar a algún centro de distribución al finalizarla.

### **Conjuntos**
* $C$: Conjunto de clientes.
* $D$: Conjunto de centros de distribución.
* $V$: Conjunto de vehículos.
* $A \subseteq (C \cup D) \times (C \cup D)$: Conjunto de arcos posibles entre nodos.

### **Parámetros**
* $cap_d$: Capacidad máxima del centro de distribución $d \in D$.
* $cap_v$: Capacidad útil de carga del vehículo $v \in V$.
* $rang_v$: Rango útil (en km) del vehículo $v \in V$.
* $dem_c$: Demanda del cliente $c \in C$.
* $dist_{ij}$: Distancia entre nodo $i$ y nodo $j$, para $(i,j) \in A$.
* $Pf$: Precio del combustible por litro (COP/l).
* $Ft$: Tarifa de flete por kilómetro recorrido (COP/km).
* $Cm$: Costo de mantenimiento por kilómetro recorrido (COP/km).
* $RC$: Rendimiento del combustible (l/km)


### **Variables de decisión**
* $x_{ijv} \in \{0,1\}$: 1 si el vehículo $v$ recorre el arco $(i,j)$, 0 en caso contrario.
* $u_{iv} \leq |(C \cup D)| - 1$: Variable auxiliar para la prevencion de sub-tours.



### **Función objetivo**
Minimizar el costo total de operación

$$\min \sum_{v \in V} \sum_{(i,j) \in A} x_{ijv} \cdot dist_{ij} \cdot (Ft + Cm + (RC \cdot Pf))$$

### **Restricciones**

Capacidad de centros de distribución:
* $\sum_{i \in C} \sum_{v \in V} x_{div}\cdot dem_c \leq cap_d \quad \forall d \in D$

Capacidad de vehículos:
* $\sum_{(i, c) \in A} x_{icv}\cdot dem_{c} \leq cap_v \quad \forall v \in V\  | \ c \in C$

Todos los clientes deben ser visitados:
* $\sum_{v \in V}\sum_{i \in C U D} x_{icv} = 1 \quad \forall c \in C$

Autonomía de vehículos:
* $\sum_{(i,j) \in A} dist_{ij} \cdot x_{ijv} \leq rang_v \quad \forall v \in V$

Un solo vehículo por salida de centro de distribucion:
* $\sum_{c \in C} \sum_{d \in D} x_{dcv} \leq 1, \forall v \in V$

Conservación del flujo para los nodos intermedios(flujo balanceado por nodo):
* $\sum_{j: (i,j) \in A} x_{ijv} - \sum_{j: (j,i) \in A} x_{jiv} = 0 \quad \forall i \in C, \forall v \in V$

Un solo vehículo por llegada al centro de distribucion:
* $\sum_{c \in C} \sum_{d \in D} x_{cdv} \leq 1, \forall v \in V$

Eliminación de Sub-Tours:
* $u_{iv} - u_{jv} + |(C \cup D)| \cdot x_{ijv} \leq |(C \cup D)| - 1 \quad \forall i,j \in C, v \in V, i \neq j\quad$

Llegada de cada vehículo a un único centro de distribución:
* $\sum_{c \in C} \sum_{d \in D} x_{cdv} \geq 1, \forall v \in V$


## **Procesamiento de Datos**

In [ ]:
vehicles1 = pd.read_csv('data/vehicles.csv')
vehicles1

,VehicleID,Capacity,Range
0,1,130,170
1,2,140,200
2,3,120,180
3,4,100,90
4,5,70,100
5,6,55,170
6,7,110,150
7,8,114,140


In [ ]:
clients1 = pd.read_csv('data/clients.csv')
clients1

,ClientID,LocationID,Demand,Longitude,Latitude
0,1,2,13,-74.098938,4.597954
1,2,3,15,-74.075571,4.687821
2,3,4,12,-74.107085,4.709494
3,4,5,15,-74.097280,4.605029
4,5,6,20,-74.164641,4.648464
5,6,7,17,-74.120838,4.662137
6,7,8,17,-74.022131,4.697499
7,8,9,20,-74.172075,4.649417
8,9,10,20,-74.156153,4.606311
9,10,11,15,-74.090411,4.557380


In [ ]:
depots1 = pd.read_csv('data/depots.csv')
depots1

,LocationID,DepotID,Longitude,Latitude
0,1,-74.153536,4.743359,NaN


## **Modelo en Pyomo**

In [ ]:
import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"
    import helper
    helper.install_idaes()
    helper.install_glpk()
    helper.install_ipopt()

centros = {
    "CD1": {"lat": 4.7110, "lon": -74.0721, "cap": 20000},
    "CD2": {"lat": 4.6050, "lon": -74.0835, "cap": 50000},
    "CD3": {"lat": 4.6700, "lon": -74.0300, "cap": 30000}
}

clientes = {
    "C1": {"lat": 4.6486, "lon": -74.0608, "demanda": 50},
    "C2": {"lat": 4.7333, "lon": -74.0700, "demanda": 80},
    "C3": {"lat": 4.7000, "lon": -74.0200, "demanda": 65}
}

vehiculos = {
    "V1": {"cap": 100, "rango": 120},
    "V2": {"cap": 80, "rango": 100},
    "V3": {"cap": 150, "rango": 150}
}

parametros = {"Pf": 15000, "Ft": 5000, "Cm": 700, "RC": 0.35}

#Usa la API de OSRM (Open Source Routing Machine)
def get_route_info(lat1, lon1, lat2, lon2):
    base_url = "http://router.project-osrm.org/route/v1/driving/"
    coordinates = f"{lon1},{lat1};{lon2},{lat2}"
    url = base_url + coordinates
    params = {
        "overview": "full",
        "geometries": "geojson"
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        if "routes" in data and data["routes"]:
            route = data["routes"][0]
            distance = route["distance"] / 1000.0
            geometry = route["geometry"]["coordinates"]
            polyline = [[coord[1], coord[0]] for coord in geometry]
            return distance, polyline
        else:
            raise Exception("No se encontró una ruta.")
    except Exception as e:
        print("Error al obtener ruta vía OSRM:", e)
        fallback_distance = haversine((lat1, lon1), (lat2, lon2))
        fallback_polyline = [[lat1, lon1], [lat2, lon2]]
        return fallback_distance, fallback_polyline

all_nodes = list(centros.keys()) + list(clientes.keys())
distancias = {}
geometries = {}

for i in all_nodes:
    for j in all_nodes:
        if i != j and not (i in centros and j in centros):
            if i in centros:
                lat_i, lon_i = centros[i]["lat"], centros[i]["lon"]
            else:
                lat_i, lon_i = clientes[i]["lat"], clientes[i]["lon"]
            if j in centros:
                lat_j, lon_j = centros[j]["lat"], centros[j]["lon"]
            else:
                lat_j, lon_j = clientes[j]["lat"], clientes[j]["lon"]
            d, polyline = get_route_info(lat_i, lon_i, lat_j, lon_j)
            distancias[(i, j)] = d
            geometries[(i, j)] = polyline

print(pd.DataFrame.from_dict(distancias, orient="index", columns=["Distancia (km)"]))

model = ConcreteModel()

# Conjuntos
model.C = Set(initialize=clientes.keys())
model.D = Set(initialize=centros.keys())
model.V = Set(initialize=vehiculos.keys())
model.A = Set(
    within=(model.D | model.C) * (model.D | model.C),
    initialize=[(i, j) for i in (set(centros.keys()) | set(clientes.keys()))
                for j in (set(centros.keys()) | set(clientes.keys()))
                if i != j and not (i in centros and j in centros)]
)

# Parámetros
model.dem_c = Param(model.C, initialize={c: clientes[c]["demanda"] for c in clientes})
model.dist_ij = Param(model.A, initialize=distancias)
model.cap_v = Param(model.V, initialize={v: vehiculos[v]["cap"] for v in vehiculos})
model.rang_v = Param(model.V, initialize={v: vehiculos[v]["rango"] for v in vehiculos})
model.cap_d = Param(model.D, initialize={d: centros[d]["cap"] for d in centros})

# Variables
model.x = Var(model.A, model.V, within=Binary)
model.u = Var(model.C, model.V, within=NonNegativeIntegers,
              bounds=(0, (len(model.C | model.D) - 1)))

# Función objetivo
def funcion_objetivo(model):
    return sum(model.x[i, j, v] * model.dist_ij[i, j] * (parametros["Ft"] + parametros["Cm"] + (parametros["RC"] * parametros["Pf"]))
               for v in model.V
               for (i, j) in model.A)
model.objetivo = Objective(rule=funcion_objetivo, sense=minimize)

# Restricciones

# 1. Capacidad de centros de distribución
def capacidad_distribucion_rule(model, d):
    return sum(model.x[d, c, v] * model.dem_c[c] for c in model.C for v in model.V) <= model.cap_d[d]
model.capacidad_distribucion = Constraint(model.D, rule=capacidad_distribucion_rule)

# 2. Capacidad de vehículos
def capacidad_vehiculos_rule(model, v):
    return sum(model.x[i, c, v] * model.dem_c[c] for (i, c) in model.A if c in model.C) <= model.cap_v[v]
model.capacidad_vehiculos = Constraint(model.V, rule=capacidad_vehiculos_rule)

# 3. Todos los clientes deben ser visitados
def visitar_clientes_rule(model, c):
    return sum(model.x[i, c, v] for v in model.V for i in (set(centros.keys()) | set(clientes.keys())) if (i, c) in model.A) == 1
model.visitar_clientes = Constraint(model.C, rule=visitar_clientes_rule)

# 4. Autonomía de vehículos
def autonomia_vehiculos_rule(model, v):
    return sum(model.dist_ij[i, j] * model.x[i, j, v] for (i, j) in model.A) <= model.rang_v[v]
model.autonomia_vehiculos = Constraint(model.V, rule=autonomia_vehiculos_rule)

# 5. Conservación del flujo
def conservacion_flujo_rule(model, i, v):
    nodos = set(centros.keys()) | set(clientes.keys())
    salida = sum(model.x[i, j, v] for j in nodos if (i, j) in model.A)
    entrada = sum(model.x[j, i, v] for j in nodos if (j, i) in model.A)
    return salida - entrada == 0 if i in model.C else salida - entrada <= 1
model.conservacion_flujo = Constraint((set(centros.keys()) | set(clientes.keys())), model.V, rule=conservacion_flujo_rule)

# 6. Eliminación de subrutas (subtours)
def subtours_elimination_rule(model, i, j, v):
    if i != j and i in model.C and j in model.C:
        return model.u[i, v] - model.u[j, v] + (len(set(centros.keys()) | set(clientes.keys()))) * model.x[i, j, v] <= (len(set(centros.keys()) | set(clientes.keys())) - 1)
    else:
        return Constraint.Skip
model.subtours_elimination = Constraint(model.C, model.C, model.V, rule=subtours_elimination_rule)

# Restricción de salida inicial: cada vehículo sale de algún centro hacia un cliente
def salida_unica_rule(model, v):
    return sum(model.x[d, j, v] for d in model.D for j in model.C if (d, j) in model.A) == 1
model.salida_unica = Constraint(model.V, rule=salida_unica_rule)

# Restricción de retorno: cada vehículo debe retornar a un centro
def retorno_centro_rule(model, v):
    return sum(model.x[i, d, v] for d in model.D for i in model.C if (i, d) in model.A) >= 1
model.retorno_centro = Constraint(model.V, rule=retorno_centro_rule)


solver = SolverFactory('glpk')
results = solver.solve(model, tee=True)

print("Asignaciones por vehículo:")
for v in model.V:
    print(f"\nVehículo {v}:")
    for (i, j) in model.A:
        if model.x[i, j, v].value is not None and model.x[i, j, v].value == 1:
            print(f"  - {i} -> {j}")


#Visualización
import folium

all_coords = [(centros[n]["lat"], centros[n]["lon"]) for n in centros] + [(clientes[n]["lat"], clientes[n]["lon"]) for n in clientes]
mean_lat = sum([lat for lat, lon in all_coords]) / len(all_coords)
mean_lon = sum([lon for lat, lon in all_coords]) / len(all_coords)

m = folium.Map(location=[mean_lat, mean_lon], zoom_start=13)

for cd in centros:
    folium.Marker(
        location=[centros[cd]["lat"], centros[cd]["lon"]],
        popup=f"Centro {cd}",
        icon=folium.Icon(color='green', icon='home', prefix='fa')
    ).add_to(m)

for c in clientes:
    folium.Marker(
        location=[clientes[c]["lat"], clientes[c]["lon"]],
        popup=f"Cliente {c}",
        icon=folium.Icon(color='blue', icon='user', prefix='fa')
    ).add_to(m)

vehicle_colors = {'V1': 'red', 'V2': 'orange', 'V3': 'purple'}

for v in model.V:
    for (i, j) in model.A:
        if model.x[i, j, v].value is not None and model.x[i, j, v].value == 1:
            polyline = geometries.get((i, j))
            if polyline:
                folium.PolyLine(
                    locations=polyline,
                    color=vehicle_colors.get(v, 'black'),
                    weight=5,
                    opacity=0.8,
                    popup=f"Ruta {i} -> {j} (Vehículo {v})"
                ).add_to(m)


m.save("ruta_optima_folium.html")
print("El mapa de rutas ha sido guardado en 'ruta_optima_folium.html'.")


--2025-05-04 05:34:36--  https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7171 (7.0K) [text/plain]
Saving to: ‘helper.py.76’

helper.py.76        100%[===================>]   7.00K  --.-KB/s    in 0s      

2025-05-04 05:34:37 (40.5 MB/s) - ‘helper.py.76’ saved [7171/7171]

IDAES found! No need to install.
Installing glpk via apt-get...
Ipopt found! No need to install.
ipopt was successfully installed
k_aug was successfuly installed
cbc was successfuly installed
clp was successfuly installed
bonmin was successfuly installed
couenne was successfuly installed
ipopt_l1 was successfuly installed
 
           Distancia (km)
(CD1, C1)          9.4502
(CD1, C2)          4.4471
(CD1, C3)       

## **Caso 1: CVRP Estándar**

In [ ]:

depot = depots1.iloc[0]
centros = {
    "CD1": {
        "lat": depot["Longitude"],
        "lon": depot["DepotID"],     #ERROR ID
        "cap": 40000
    }
}

centros

{'CD1': {'lat': np.float64(4.743359),
  'lon': np.float64(-74.153536),
  'cap': 40000}}

In [ ]:
base_cap = vehicles1.iloc[0]["Capacity"]
base_rang = vehicles1.iloc[0]["Range"]

vehiculos = {f"V{v}": {"cap": base_cap, "rango": base_rang}
             for v in vehicles1["VehicleID"]}

vehiculos

{'V1': {'cap': np.int64(130), 'rango': np.int64(170)},
 'V2': {'cap': np.int64(130), 'rango': np.int64(170)},
 'V3': {'cap': np.int64(130), 'rango': np.int64(170)},
 'V4': {'cap': np.int64(130), 'rango': np.int64(170)},
 'V5': {'cap': np.int64(130), 'rango': np.int64(170)},
 'V6': {'cap': np.int64(130), 'rango': np.int64(170)},
 'V7': {'cap': np.int64(130), 'rango': np.int64(170)},
 'V8': {'cap': np.int64(130), 'rango': np.int64(170)}}

In [ ]:
clientes = {
    row["ClientID"]: {"lat": row["Latitude"], "lon": row["Longitude"], "demanda": row["Demand"]}
    for _, row in clients1.iterrows()
}
clientes

{np.float64(1.0): {'lat': np.float64(4.59795431125545),
  'lon': np.float64(-74.09893796560621),
  'demanda': np.float64(13.0)},
 np.float64(2.0): {'lat': np.float64(4.687820646838871),
  'lon': np.float64(-74.07557103763986),
  'demanda': np.float64(15.0)},
 np.float64(3.0): {'lat': np.float64(4.70949446000624),
  'lon': np.float64(-74.10708524062704),
  'demanda': np.float64(12.0)},
 np.float64(4.0): {'lat': np.float64(4.605029068682624),
  'lon': np.float64(-74.09727965657427),
  'demanda': np.float64(15.0)},
 np.float64(5.0): {'lat': np.float64(4.648463876533332),
  'lon': np.float64(-74.16464148202755),
  'demanda': np.float64(20.0)},
 np.float64(6.0): {'lat': np.float64(4.662137416953968),
  'lon': np.float64(-74.12083799988112),
  'demanda': np.float64(17.0)},
 np.float64(7.0): {'lat': np.float64(4.697499030379109),
  'lon': np.float64(-74.02213076607309),
  'demanda': np.float64(17.0)},
 np.float64(8.0): {'lat': np.float64(4.649416884236942),
  'lon': np.float64(-74.17207549744

## **Caso 2: Múltiples Centros de Distribución**

## **Caso 3: Escenario Realista**